# greedy

In [84]:
import math
import numpy as np
import pandas as pd

In [85]:
df1 = pd.read_csv("../Dataset/ShipCase1.csv")
df2 = pd.read_csv("../Dataset/ShipCase2.csv")
df3 = pd.read_csv("../Dataset/ShipCase3.csv")
df4 = pd.read_csv("../Dataset/ShipCase4.csv")
df5 = pd.read_csv("../Dataset/ShipCase5.csv")
df6 = pd.read_csv("../Dataset/ShipCase6.csv")

In [86]:
# The given test set is not very good in my oponion, so I geenrate a new test set here: it cover many edges cases
# test data
test_data = np.array([
    [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
    [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
    [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
    [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
    [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
    [  0,   0,   0,  50,   0,   0,   0,   0,   0,   0,   0,   0],
    [ -1,   0,   0,  82,  50,   0,   0,   0,   0,   0,   0,  -1],
    [ -1,  -1,  20,  40,  20,  20,   0,   0,   0,   0,  -1,  -1],
    [ -1,  -1, 100, 334, 333, 333,   0,   0,   0,   0,  -1,  -1]
], dtype=int)
test_data

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,  50,   0,   0,   0,   0,   0,   0,   0,   0],
       [ -1,   0,   0,  82,  50,   0,   0,   0,   0,   0,   0,  -1],
       [ -1,  -1,  20,  40,  20,  20,   0,   0,   0,   0,  -1,  -1],
       [ -1,  -1, 100, 334, 333, 333,   0,   0,   0,   0,  -1,  -1]])

1. Define the representation and initial state

*Note that crane located at [0, 8] (by slides, just above [1, 8])*

In [87]:
# This is for given test set only because row 1 start from bottom- need to flip it during computing.
# Store mainfest: NAN as -1, 0 as empty, >0 as occupied, number are weight of the container
def create_grid(path):
    df = pd.read_csv(path)
    nrows = int(df["row"].max()) + 1
    ncols = int(df["column"].max())
    grid = np.zeros((nrows, ncols), dtype=int)

    for _, rec in df.iterrows():
        r = nrows - int(rec["row"])
        c = int(rec["column"]) - 1
        desc = str(rec["description"]).strip().upper()

        if desc == "NAN":
            grid[r, c] = -1
        elif desc == "UNUSED":
            grid[r, c] = 0
        else:
            grid[r, c] = int(rec["weight"])
    return grid

# grid6 = create_grid("../Dataset/ShipCase6.csv")
# grid6

2. Define goal state

In [88]:
def left_weight(grid):
    return int(np.clip(grid[1:9, 0:6], 0, None).sum())

def right_weight(grid):
    return int(np.clip(grid[1:9, 6:12], 0, None).sum())

def which_side_is_heavier(grid):
    left = left_weight(grid)
    right = right_weight(grid)
    return (0, 6) if left > right else (6, 12)

def goal_test(grid, tol=0.10):
    total = left_weight(grid) + right_weight(grid)
    return True if total == 0 else abs(left_weight(grid) - right_weight(grid)) <= tol * total

# goal_test(grid6)
# goal_test(test_data)

In [89]:
def get_set_data(data):
    left = left_weight(data)
    right = right_weight(data)
    goal = goal_test(data)
    total = left + right
    needed = math.ceil((abs(left-right) - 0.1*total) / 2)

    print("Left: ", left, "\nRight: ", right)
    print("Total: ", total)
    print("Diff: ", abs(left-right))
    print("Goal: ", goal)
    print("Target: ", 0.1*total) # weight diff small or equal to this number
    print("Needed: ", needed)

get_set_data(test_data)

Left:  1382 
Right:  0
Total:  1382
Diff:  1382
Goal:  False
Target:  138.20000000000002
Needed:  622


3. Define the operators (What to move)

In [90]:
# what to move: determine which container to move
def determine_container_to_move(grid):
    # compute which side is heavier
    left_sum = left_weight(grid)
    right_sum = right_weight(grid)

    low, high = (0, 6) if left_sum > right_sum else (6, 12)
    
    # comopute the needed weight to move
    total = left_sum + right_sum
    diff = abs(left_sum - right_sum)
    target = total * 0.1
    needed = math.ceil((diff - target) / 2)
    
    # find the container to move
    candidates = [((row, col), int(grid[row, col]))
                    for row in range(grid.shape[0])
                    for col in range(low, high)
                    if int(grid[row, col]) > 0]
    
    # a. check if can do done by move single container
    for pos, weight in candidates:
        if weight >= needed:
            return [(pos, weight)], needed
        # else: continue
    
    # b. otherwise, try combinations containers by getting the very top container
    top_sorted = sorted(candidates, key=lambda x: x[0][0])
    current_combination = []
    current_weight = 0

    for pos, weight in top_sorted:
        current_combination.append((pos, weight))
        current_weight += weight
        if current_weight >= needed:
            return current_combination, needed
    
    return candidates, needed

candidates, needed = determine_container_to_move(test_data)
candidates, needed


([((5, 3), 50),
  ((6, 3), 82),
  ((6, 4), 50),
  ((7, 2), 20),
  ((7, 3), 40),
  ((7, 4), 20),
  ((7, 5), 20),
  ((8, 2), 100),
  ((8, 3), 334)],
 622)

After determine which container need to move, now it's time to move them (greedy move):

4. Search algorithm (Actual Operation)

    Since we picked very top container to move, we don't need to check if any container above it. We can just start moving. 

    - Slide horizontally to the right: grid to the right might be blocked, move up until right cell is empty; then move right; repeat if same silution happen. 
    - Drop down. Once on the target side, drop it down. To save steps, can drop on columns that nearest the border. 

In [93]:
def move_container_another_side(grid, container_to_move):

    moves = []

    # check which side is heavier
    heavier_side = which_side_is_heavier(grid)

    if heavier_side == (0, 6):
        target_cols = range(11, 5, -1)
    else:
        target_cols = range(0, 6)

    # start moving
    for (row, col), weight in container_to_move:
        destination = None

        for col in target_cols:                         # columns 12→7 (indexes)
            for row in range(grid.shape[0]-1, -1, -1):  # bottom → top
                if grid[row, col] == 0:                 # avaiable
                    destination = (row, col)
                if grid[row, col] == -1:                # not avaiable
                    continue

        if destination is None:
            raise ValueError("No destination found")
        
        # perform move
        destination_row, destination_col = destination
        grid[destination_row, destination_col] = grid[row, col]
        grid[row, col] = 0
        moves.append(((row, col), destination))

    return grid, moves
        
# test
grid, moves = move_container_another_side(test_data, candidates)
print(grid)
print(moves)


[[  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  50   0   0   0   0   0   0   0   0]
 [ -1   0   0  82  50   0   0   0   0   0   0  -1]
 [ -1  -1  20  40  20  20   0   0   0   0  -1  -1]
 [ -1  -1 100 334 333 333   0   0   0   0  -1  -1]]
[((0, 6), (0, 6)), ((0, 6), (0, 6)), ((0, 6), (0, 6)), ((0, 6), (0, 6)), ((0, 6), (0, 6)), ((0, 6), (0, 6)), ((0, 6), (0, 6)), ((0, 6), (0, 6)), ((0, 6), (0, 6))]


In [92]:
# check this on main
# if (!goal_test(grid)):
#     raise ValueError("The initial grid does not meet the balance goal.")